In [1]:
import pandas as pd

frames = []

for year in range(2016,2024):
    url="https://fixturedownload.com/download/epl-%s-GMTStandardTime.csv" % year
    print(url)
    frame = pd.read_csv(url)
    frame['Season']=year
    frames.append(frame)
df = pd.concat(frames)
df.head()

https://fixturedownload.com/download/epl-2016-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2017-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2018-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2019-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2020-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2021-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2022-GMTStandardTime.csv
https://fixturedownload.com/download/epl-2023-GMTStandardTime.csv


,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,Season
0,1,1,13/08/2016 12:30,KCOM Stadium,Hull,Leicester,2 - 1,2016
1,2,1,13/08/2016 15:00,Turf Moor,Burnley,Swansea,0 - 1,2016
2,3,1,13/08/2016 15:00,Selhurst Park,Crystal Palace,West Brom,0 - 1,2016
3,4,1,13/08/2016 15:00,Goodison Park,Everton,Spurs,1 - 1,2016
4,5,1,13/08/2016 15:00,Riverside Stadium,Middlesbrough,Stoke,1 - 1,2016


In [2]:
df.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,Season
375,376,38,19/05/2024 16:00,Selhurst Park,Crystal Palace,Aston Villa,NaN,2023
376,377,38,19/05/2024 16:00,Anfield,Liverpool,Wolves,NaN,2023
377,378,38,19/05/2024 16:00,Kenilworth Road,Luton,Fulham,NaN,2023
378,379,38,19/05/2024 16:00,Etihad Stadium,Man City,West Ham,NaN,2023
379,380,38,19/05/2024 16:00,Bramall Lane,Sheffield Utd,Spurs,NaN,2023


In [3]:
# Filter out the matches that haven't been played yet.
df = df[pd.notnull(df.Result)]
df.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,Season
279,275,28,11/03/2024 20:00,Stamford Bridge,Chelsea,Newcastle,3 - 2,2023
281,283,29,16/03/2024 15:00,Turf Moor,Burnley,Brentford,2 - 1,2023
283,287,29,16/03/2024 15:00,Kenilworth Road,Luton,Nottingham Forest,1 - 1,2023
286,286,29,16/03/2024 17:30,Craven Cottage,Fulham,Spurs,3 - 0,2023
288,289,29,17/03/2024 14:00,London Stadium,West Ham,Aston Villa,1 - 1,2023


In [4]:
import re

def parse_scores(result):
    return list(map(lambda x:int(x), re.findall(r'\b\d+\b',result) ))

def parse_home_score(row):
    return parse_scores(row['Result'])[0]


def parse_away_score(row): 
    return parse_scores(row['Result'])[1]


df['Home Score']=df.apply(parse_home_score, axis=1)
df['Away Score']=df.apply(parse_away_score, axis=1)

df.head()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,Season,Home Score,Away Score
0,1,1,13/08/2016 12:30,KCOM Stadium,Hull,Leicester,2 - 1,2016,2,1
1,2,1,13/08/2016 15:00,Turf Moor,Burnley,Swansea,0 - 1,2016,0,1
2,3,1,13/08/2016 15:00,Selhurst Park,Crystal Palace,West Brom,0 - 1,2016,0,1
3,4,1,13/08/2016 15:00,Goodison Park,Everton,Spurs,1 - 1,2016,1,1
4,5,1,13/08/2016 15:00,Riverside Stadium,Middlesbrough,Stoke,1 - 1,2016,1,1


In [5]:
# Extract unique team names from column B
team_names = df['Home Team'].unique()
df.describe()

,Match Number,Round Number,Season,Home Score,Away Score
count,2943.000000,2943.000000,2943.000000,2943.000000,2943.000000
mean,185.851172,19.115189,2019.384642,1.552497,1.262317
std,108.300453,10.877445,2.237784,1.328150,1.222313
min,1.000000,1.000000,2016.000000,0.000000,0.000000
25%,92.500000,10.000000,2017.000000,1.000000,0.000000
50%,184.000000,19.000000,2019.000000,1.000000,1.000000
75%,276.500000,28.000000,2021.000000,2.000000,2.000000
max,380.000000,38.000000,2023.000000,9.000000,9.000000


In [6]:
# Check for missing values in the 'Home Team' column
missing_teams = df[df['Home Team'].isna()]

# Display missing teams, if any
if not missing_teams.empty:
    print("Missing Teams:")
    print(missing_teams['Home Team'])
else:
    print("No missing teams.")

No missing teams.


In [7]:
# Dictionary to store data for each team
team_data = {}


for team in team_names:
    # Filter rows for the teams as Home Team
    home_team_rows = df[df['Home Team'] == team]

    # Calculate average goals for and against as Home Team
    avg_home_goals_for = home_team_rows['Home Score'].mean()
    avg_away_goals_against = home_team_rows['Away Score'].mean()

    # Filter rows for the specific team as Away Team
    away_team_rows = df[df['Away Team'] == team]

    # Calculate average goals for and against as Away Team
    avg_away_goals_for = away_team_rows['Away Score'].mean()
    avg_home_goals_against = away_team_rows['Home Score'].mean()

    # Calculate average goals for and against for each team
    OFF_rating = (avg_home_goals_for + avg_away_goals_for) / 2
    DEF_rating = (avg_home_goals_against + avg_away_goals_against) / 2

    # Store  data in the dictionary
    team_data[team] = {
        'Home Goals For': avg_home_goals_for,
        'Away Goals For': avg_away_goals_for,
        'Home Goals Against': avg_home_goals_against,
        'Away Goals Against': avg_away_goals_against,
        'OFF Rating': OFF_rating,
        'DEF Rating': DEF_rating
    }

# Display all data for each team
print("Team Data:")
for team, data in team_data.items():
    print(f"{team}:")
    print(f"  Home Goals For: {data['Home Goals For']:.5f}")
    print(f"  Away Goals For: {data['Away Goals For']:.5f}")
    print(f"  Home Goals Against: {data['Home Goals Against']:.5f}")
    print(f"  Away Goals Against: {data['Away Goals Against']:.5f}")
    print(f"  OFF Rating: {data['OFF Rating']:.2f}")
    print(f"  DEF Rating: {data['DEF Rating']:.2f}")
    print()


Team Data:
Hull:
  Home Goals For: 1.47368
  Away Goals For: 0.47368
  Home Goals Against: 2.36842
  Away Goals Against: 1.84211
  OFF Rating: 0.97
  DEF Rating: 2.11

Burnley:
  Home Goals For: 1.06202
  Away Goals For: 0.95312
  Home Goals Against: 1.59375
  Away Goals Against: 1.38760
  OFF Rating: 1.01
  DEF Rating: 1.49

Crystal Palace:
  Home Goals For: 1.18367
  Away Goals For: 1.13605
  Home Goals Against: 1.65306
  Away Goals Against: 1.27211
  OFF Rating: 1.16
  DEF Rating: 1.46

Everton:
  Home Goals For: 1.40136
  Away Goals For: 1.02721
  Home Goals Against: 1.60544
  Away Goals Against: 1.21088
  OFF Rating: 1.21
  DEF Rating: 1.41

Middlesbrough:
  Home Goals For: 0.89474
  Away Goals For: 0.52632
  Home Goals Against: 1.57895
  Away Goals Against: 1.21053
  OFF Rating: 0.71
  DEF Rating: 1.39

Southampton:
  Home Goals For: 1.16541
  Away Goals For: 1.09023
  Home Goals Against: 1.79699
  Away Goals Against: 1.48872
  OFF Rating: 1.13
  DEF Rating: 1.64

Man City:
  Hom

In [8]:
# Display just OFF & DEF for each team
print("Team Data:")
for team, data in team_data.items():
    print(f"{team}:")
    print(f"  OFF Rating: {data['OFF Rating']:.5f}")
    print(f"  DEF Rating: {data['DEF Rating']:.5f}")
    print()

Team Data:
Hull:
  OFF Rating: 0.97368
  DEF Rating: 2.10526

Burnley:
  OFF Rating: 1.00757
  DEF Rating: 1.49067

Crystal Palace:
  OFF Rating: 1.15986
  DEF Rating: 1.46259

Everton:
  OFF Rating: 1.21429
  DEF Rating: 1.40816

Middlesbrough:
  OFF Rating: 0.71053
  DEF Rating: 1.39474

Southampton:
  OFF Rating: 1.12782
  DEF Rating: 1.64286

Man City:
  OFF Rating: 2.45578
  DEF Rating: 0.82653

Bournemouth:
  OFF Rating: 1.25688
  DEF Rating: 1.77064

Arsenal:
  OFF Rating: 1.88435
  DEF Rating: 1.18367

Chelsea:
  OFF Rating: 1.70005
  DEF Rating: 1.10870

Man Utd:
  OFF Rating: 1.63265
  DEF Rating: 1.12245

Stoke:
  OFF Rating: 1.00000
  DEF Rating: 1.63158

Swansea:
  OFF Rating: 0.96053
  DEF Rating: 1.65789

Spurs:
  OFF Rating: 1.88435
  DEF Rating: 1.14966

Watford:
  OFF Rating: 1.08421
  DEF Rating: 1.74737

West Brom:
  OFF Rating: 0.95614
  DEF Rating: 1.60526

Leicester:
  OFF Rating: 1.51504
  DEF Rating: 1.46241

Sunderland:
  OFF Rating: 0.76316
  DEF Rating: 1.81

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
from datetime import date


In [11]:
#gather elo ratings 
elo_rating_url="http://api.clubelo.com/%s" % date.today()
print("Downloading ELO Ratings from %s" % elo_rating_url)
date.today()
elo_df = pd.read_csv(elo_rating_url)
elo_df.head()

,Rank,Club,Country,Level,Elo,From,To
0,1.0,Man City,ENG,1,2052.299072,2024-03-15,2024-03-31
1,2.0,Real Madrid,ESP,1,1971.240479,2024-03-18,2024-03-31
2,3.0,Inter,ITA,1,1970.814087,2024-03-18,2024-04-01
3,4.0,Liverpool,ENG,1,1967.693604,2024-03-17,2024-03-31
4,5.0,Arsenal,ENG,1,1956.634888,2024-03-15,2024-03-31


In [14]:
#refine to only epl club ratings
elo_df = elo_df[elo_df["Country"]=="ENG"]
elo_df['Team'] = elo_df['Club']
elo_df['ELO Rating'] = elo_df['Elo']

elo_df.head()

,Rank,Club,Country,Level,Elo,From,To,Team,ELO Rating
0,1.0,Man City,ENG,1,2052.299072,2024-03-15,2024-03-31,Man City,2052.299072
3,4.0,Liverpool,ENG,1,1967.693604,2024-03-17,2024-03-31,Liverpool,1967.693604
4,5.0,Arsenal,ENG,1,1956.634888,2024-03-15,2024-03-31,Arsenal,1956.634888
11,12.0,Tottenham,ENG,1,1841.693726,2024-03-18,2024-03-30,Tottenham,1841.693726
12,13.0,Aston Villa,ENG,1,1836.807983,2024-03-18,2024-03-30,Aston Villa,1836.807983


In [ ]:
#calculating the 1x2 prob from correct score, using a poisson dist to calc each correct score probability for each team

In [ ]:
def calculate_poisson_probability(home_xg, away_xg, home_goals, away_goals):
    home_prob = poisson.pmf(home_goals, home_xg)
    away_prob = poisson.pmf(away_goals, away_xg)
    return home_prob * away_prob

def simulate_poisson_distribution(home_xg, away_xg):
    max_goals = 10  # Maximum number of goals to consider
    score_matrix = np.zeros((max_goals, max_goals))

    for home_goals in range(max_goals):
        for away_goals in range(max_goals):
            score_matrix[home_goals][away_goals] = calculate_poisson_probability(home_xg, away_xg, home_goals, away_goals)

    return score_matrix



In [53]:
import numpy as np
from scipy.stats import poisson

# highest number of goals in our sample = 9
number_of_goals = 9

home_team_xg = 2.35229  # calculated from a seperate source
away_team_xg = 1.74208  # calculated from a seperate source

#simulate the poisson dist for each goal
def calculate_poisson_probability(home_xg, away_xg, home_goals, away_goals):
    home_prob = poisson.pmf(home_goals, home_xg)
    away_prob = poisson.pmf(away_goals, away_xg)
    return home_prob * away_prob

def simulate_poisson_distribution(home_xg, away_xg):
    score_matrix = np.zeros((number_of_goals, number_of_goals))

    for home_goals in range(number_of_goals):
        for away_goals in range(number_of_goals):
            score_matrix[home_goals][away_goals] = calculate_poisson_probability(home_xg, away_xg, home_goals, away_goals)

    return score_matrix

In [54]:
# build the above score matrix as a grid as apposed to a list
def print_result_grid(result_matrix):
    print("  |", end="")
    for i in range(number_of_goals):
        print(f"  {i}   |", end="")
    print("\n--+------+------+------+------+------+------+------+------+------+")

    for home_goals in range(number_of_goals):
        print(f"{home_goals} |", end="")
        for away_goals in range(number_of_goals):
            print(f" {result_matrix[home_goals][away_goals]:.2f} |", end="")
        print()

In [55]:
#add up the probs to get a 1x2 probability for the fixture
def calculate_win_draw_probabilities(result_matrix):
    home_win_prob = np.sum(np.tril(result_matrix, -1))  # Sum the lower triangular part for home win
    away_win_prob = np.sum(np.triu(result_matrix, 1))   # Sum the upper triangular part for away win
    draw_prob = np.sum(np.diag(result_matrix))         # Sum the diagonal for draw

    return home_win_prob, away_win_prob, draw_prob


In [56]:
result_matrix = simulate_poisson_distribution(home_team_xg, away_team_xg)

In [57]:
# Display the result matrix as a grid
print_result_grid(result_matrix)

  |  0   |  1   |  2   |  3   |  4   |  5   |  6   |  7   |  8   |
--+------+------+------+------+------+------+------+------+------+
0 | 0.02 | 0.03 | 0.03 | 0.01 | 0.01 | 0.00 | 0.00 | 0.00 | 0.00 |
1 | 0.04 | 0.07 | 0.06 | 0.03 | 0.02 | 0.01 | 0.00 | 0.00 | 0.00 |
2 | 0.05 | 0.08 | 0.07 | 0.04 | 0.02 | 0.01 | 0.00 | 0.00 | 0.00 |
3 | 0.04 | 0.06 | 0.05 | 0.03 | 0.01 | 0.00 | 0.00 | 0.00 | 0.00 |
4 | 0.02 | 0.04 | 0.03 | 0.02 | 0.01 | 0.00 | 0.00 | 0.00 | 0.00 |
5 | 0.01 | 0.02 | 0.02 | 0.01 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |
6 | 0.00 | 0.01 | 0.01 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |
7 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |
8 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |


In [58]:
# Calculate and print win and draw probabilities
home_win_prob, away_win_prob, draw_prob = calculate_win_draw_probabilities(result_matrix)

print("\nWin Probabilities:")
print(f"\nHome Win Probability: {home_win_prob:.2f}")
print(f"Away Win Probability: {away_win_prob:.2f}")
print(f"Draw Probability: {draw_prob:.2f}")

#calculate and print raw prices
raw_home_price = 1 / home_win_prob
raw_draw_price = 1 / draw_prob
raw_away_price = 1 / away_win_prob

print("\nRaw Prices:")
print(f"\nHome Price : {raw_home_price:.2f}")
print(f"Draw Price : {raw_draw_price:.2f}")
print(f"Away Price : {raw_away_price:.2f}")

#calculate and print MOU prices - 3% margin
mou_home_price = 1 / (home_win_prob*1.03)
mou_draw_price = 1 / (draw_prob*1.03)
mou_away_price = 1 / (away_win_prob*1.03)

print("\nMOU Prices:")
print(f"\nHome Price : {mou_home_price:.2f}")
print(f"Draw Price : {mou_draw_price:.2f}")
print(f"Away Price : {mou_away_price:.2f}")


Win Probabilities:

Home Win Probability: 0.52
Away Win Probability: 0.29
Draw Probability: 0.20

Raw Prices:

Home Price : 1.94
Draw Price : 5.09
Away Price : 3.49

MOU Prices:

Home Price : 1.88
Draw Price : 4.94
Away Price : 3.39
